# Grasp annotation example

In [ ]:
from pathlib import Path
from typing import List

import numpy as np
import open3d as o3d
import rerun as rr
from airo_camera_toolkit.point_clouds.conversions import open3d_to_point_cloud, point_cloud_to_open3d
from airo_camera_toolkit.point_clouds.operations import filter_point_cloud
from airo_models.primitives.mesh import mesh_urdf_path
from airo_typing import HomogeneousMatrixType, JointConfigurationType
from cloth_tools.bounding_boxes import BBOX_CLOTH_IN_THE_AIR, bbox_to_mins_and_sizes
from cloth_tools.dataset.format import load_competition_input_sample
from cloth_tools.drake.building import add_meshcat_to_builder, finish_build
from cloth_tools.drake.scenes import add_dual_ur5e_and_table_to_builder
from cloth_tools.drake.visualization import add_meshcat_triad, publish_dual_arm_joint_path, publish_ik_solutions
from cloth_tools.ompl.dual_arm_planner import DualArmOmplPlanner
from cloth_tools.point_clouds.operations import filter_and_crop_point_cloud
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from ur_analytic_ik import ur5e
import matplotlib.pyplot as plt
from cloth_tools.dataset.format import load_competition_input_sample
from typing import Tuple
from airo_camera_toolkit.utils.image_converter import ImageConverter
from airo_typing import OpenCVIntImageType
import cv2

data_dir = Path("../data")
dataset_dir = data_dir / "competition_dev"

In [ ]:
sample = load_competition_input_sample(dataset_dir, sample_index=0)

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(sample.image_left);

In [ ]:
confidence_map = sample.confidence_map
point_cloud_in_camera = sample.point_cloud

pcd_in_camera = point_cloud_to_open3d(point_cloud_in_camera) # X_C_PC, need X_W_C

# TODO change how camera_pose is saved
# TODO clarify somewhere what the world frame is to simplify all these transformations
camera_pose = sample.camera_pose # X_LCB_C (camera pose in the left-arm base frame)
X_LCB_C = camera_pose

y_distance = 0.45
X_W_L = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi / 2]), p=[0, y_distance, 0]).GetAsMatrix4()
X_W_R = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi / 2]), p=[0, -y_distance, 0]).GetAsMatrix4()
X_CB_B = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi]), p=[0, 0, 0]).GetAsMatrix4()
X_LCB_W = X_CB_B @ np.linalg.inv(X_W_L)
X_RCB_W = X_CB_B @ np.linalg.inv(X_W_R)
X_W_C = np.linalg.inv(X_LCB_W) @ X_LCB_C

pcd = pcd_in_camera.transform(X_W_C) # transform to world frame
point_cloud = open3d_to_point_cloud(pcd)

confidence_threshold = 1.0
confidence_mask = (confidence_map <= confidence_threshold).reshape(-1)  # Threshold and flatten
point_cloud_filtered = filter_point_cloud(point_cloud, confidence_mask)

In [ ]:
pcd_filtered = point_cloud_to_open3d(point_cloud_filtered)

resolution = sample.camera_resolution
intrinsics = sample.camera_intrinsics.astype(np.float32)

X = np.array([1, 0, 0])
Z = np.array([0, 0, -1])
Y = np.cross(Z, X)
orientation = np.column_stack([X, Y, Z])

position = np.array([0, 0, 2.0])

camera_pose_top_down = np.identity(4)
camera_pose_top_down[:3, :3] = orientation
camera_pose_top_down[:3, 3] = position

# pcd_in_camera.point.positions = o3d.core.Tensor(pcd_in_camera.point.positions.numpy().astype(np.float32))


camera_pose_top_down = camera_pose_top_down.astype(np.float32)

# rgbd_image = pcd_filtered.project_to_rgbd_image(*resolution, sample.camera_intrinsics, extrinsics=np.linalg.inv(camera_pose_top_down))

In [ ]:
rr.init("Grasp Annotation - Point cloud", spawn=True)

rr_point_cloud = rr.Points3D(positions=point_cloud_filtered.points, colors=point_cloud_filtered.colors)
rr.log("world/point_cloud", rr_point_cloud)

In [ ]:
line_type = cv2.LINE_AA
point_size = 15
point_thickness = 2

window_frontal = "Grasp Annotation - Frontal view"
window_topdown = "Grasp Annotation - Topdown view"
windows = [window_frontal, window_topdown]

h = 600
w = int(1.5 * h)
cv2.namedWindow(window_frontal, cv2.WINDOW_NORMAL)
cv2.namedWindow(window_topdown, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_frontal, w, h)
cv2.resizeWindow(window_topdown, h, h)
cv2.moveWindow(window_frontal, 0, 0)
cv2.moveWindow(window_topdown, w + 80, 0)

clicked_point = {
    window_frontal: None,
    window_topdown: None
}

def callback_frontal(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Frontal view: click at ({x}, {y})")
        clicked_point[window_frontal] = (x, y)
    if event == cv2.EVENT_RBUTTONDOWN:
        print(f"Frontal view: removed click.")
        clicked_point[window_frontal] = None

def callback_topdown(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Topdown view: click at ({x}, {y})")
        clicked_point[window_topdown] = (x, y)
    if event == cv2.EVENT_RBUTTONDOWN:
        print(f"Topdown view: removed click.")
        clicked_point[window_topdown] = None

def draw_point(image: OpenCVIntImageType, point: tuple[int, int], color: tuple[int, int, int] = (0, 255, 0)):
    cv2.circle(image, point, point_size, color, point_thickness, line_type)


cv2.setMouseCallback(window_frontal, callback_frontal)
cv2.setMouseCallback(window_topdown, callback_topdown)

while True:
    image = ImageConverter.from_numpy_int_format(sample.image_left).image_in_opencv_format

    image_frontal = image.copy()
    image_topdown = image.copy()

    if clicked_point[window_frontal] is not None:
        draw_point(image_frontal, clicked_point[window_frontal])

    if clicked_point[window_topdown] is not None:
        draw_point(image_topdown, clicked_point[window_topdown])

    cv2.imshow(window_frontal, image_frontal)
    cv2.imshow(window_topdown, image_topdown)
    key = cv2.waitKey(10)
    if key== ord('q'):
        cv2.destroyAllWindows()
        break